In [3]:
import sys,re,time,random
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import lxml.html
import nest_asyncio
import requests
import aiohttp
import asyncio
#import geocoder
#from fake_useragent import UserAgent

In [4]:
nest_asyncio.apply()

# Store url

In [5]:
async def fetch(session, url):
    #headers = {'User-Agent': UserAgent().random}
    #async with session.get(url,proxy=proxy) as html:
    async with session.get(url) as html:
        return await html.text()

In [6]:
async def parser(html):
    soup=bs(html,'lxml')
    urls_=soup.select('li[class="item_wrap"]>a[class="_link"]')
    #時給
    for i in range(len(urls_)):
        urls.append("https://www.sej.co.jp/"+re.findall(r"href=\"(.+);jsessionid",str(urls_[i]))[0])

In [7]:
async def download(url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url) 
        await parser(html)

In [ ]:
urls=[]
main_url="https://www.sej.co.jp/arbeit/recruitment/jobfind-pc/area/HokkaidoTohoku/Hokkaido?page="
#ページ数とる
html=requests.get(main_url)
soup=bs(html.text,'lxml')
page_sum=soup.select('span[class="num_wrap"]>span[class="_num _orange"]')
page_=int(re.findall(r"\d+",str(page_sum))[0])
page=round(page_/20)+1

#非同期ループ
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(download(main_url+str(i))) for i in range(1,page)]
tasks = asyncio.gather(*tasks)
loop.run_until_complete(tasks)

urls=set(urls)

# Store Address

In [7]:
async def parser_ads(html):
    soup=bs(html,'lxml')
    #time.sleep(random.uniform(0.5,1.5))
    #店舗名
    name_=soup.select('span[class="main"]')
    if name_:
        name1=re.findall(r"<br/>(.+)<",str(name_))[0]
        name.append(name1.replace(" ",""))
        #時給
        sl=soup.select('ul[class="infolist"] > li')[0]
        salary_=re.findall(r"(\d*\,*\d+)\s円",str(sl))
        salary.append(min([int(i.replace(",","")) for i in salary_]))
        #住所
        address=soup.select('ul[class="infolist"] > li')[1].p
        ads.append(re.findall(r"\s*(.+)\s*</p>",str(address))[0].replace(" ",""))

In [8]:
async def download_ads(url):
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, url)
        await parser_ads(html)

In [ ]:
ads=[]
name=[]
salary=[]
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(download_ads(url)) for url in urls]
tasks = asyncio.gather(*tasks)
loop.run_until_complete(tasks)

In [17]:
shop=["セブンイレブン" for i in range(len(name))]
shop1=pd.DataFrame(shop)
name1=pd.DataFrame(name)
ads1=pd.DataFrame(ads)
salary1=pd.DataFrame(salary)

In [22]:
len(urls)

796

In [19]:
cc=pd.concat([shop1,name1,ads1,salary1],axis=1)
cc.columns=["shop","name","address","salary"]
cc.to_pickle(r"ce.pkl")

# Store latitude and longitude, business hours

In [ ]:
lat=[]
lng=[]
tm=[]
main_url="https://www.google.com/maps/"
options = Options()
driver = webdriver.Chrome(options = options)
driver.get(main_url)
for i in tqdm(range(len(name))):
    #google map 検索-入力
    driver.find_element_by_id('searchboxinput').send_keys(name[i])
    driver.find_element_by_id('searchbox-searchbutton').click()
    time.sleep(3)
    lat_=re.findall(r"3d(\d+\.*\d+)!4d",driver.current_url)[0]
    lng_=re.findall(r".+4d(.+)$",driver.current_url)[0]
    lat.append(lat_)
    lng.append(lng_)
    
    tm.append(driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[12]/div[1]/span[2]/span[2]').text)
    
    driver.find_element_by_id('searchboxinput').clear()
driver.quit()


# Data storage

In [ ]:
name1=pd.DataFrame(name)
ads1=pd.DataFrame(ads)
tm1=pd.DataFrame(tm)
salary1==pd.DataFrame(salary)
lat1=pd.DataFrame(lat)
lng1=pd.DataFrame(lng)

In [ ]:
cc=pd.concat([name1,ads1,tm1,salary1,lat1,lng1],axis=1)
cc.columns=["name","address","business_hours","salary","lat","lng"]
cc.to_csv(r"711.csv",encoding="utf_8_sig",index=False)